Package imports

In [2]:
import gc
import math
import os
import time
from datetime import datetime, timedelta

import netCDF4 as nc
import numpy as np
import pandas as pd
from geopy import Point
from geopy.distance import great_circle
from scipy.spatial import cKDTree
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import ElasticNet, Ridge, Lasso, BayesianRidge
from xgboost import XGBRegressor
import lightgbm as lgb

from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
import numpy as np
import os

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
import optuna
from scipy.stats import randint, uniform
import glob
from haversine import haversine
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

Function to pre-process spatial data

In [3]:
# Precompute the KDTree and valid_time differences
def precompute_kdtree_and_time_diffs(uwnd_nc_file_path):
    try:
        print("Precomputing KDTree and time differences...")
        # Load the NetCDF file
        ds = nc.Dataset(uwnd_nc_file_path)

        # Extract the valid_time, latitudes, and longitudes from the NetCDF file
        valid_time = ds.variables['valid_time'][:]  # Assuming 'valid_time' is the variable name for time
        latitudes = ds.variables['latitude'][:]
        longitudes = ds.variables['longitude'][:]

        # Convert valid_time from seconds since 1970-01-01 to datetime
        base_time = datetime(1970, 1, 1)
        valid_time_dt = np.array([base_time + timedelta(seconds=int(ts)) for ts in valid_time], dtype='datetime64[ns]')

        # Create a KDTree for fast spatial lookup
        lat_lon_pairs = np.array([(lat, lon) for lat in latitudes for lon in longitudes])
        tree = cKDTree(lat_lon_pairs)

        print("KDTree and time differences precomputed successfully.")
        return tree, valid_time_dt, latitudes, longitudes, lat_lon_pairs
    except Exception as e:
        print(f"Error precomputing KDTree and time differences: {e}")
        raise

uwnd_nc_file_path = '../data/raw/reanalyses/ERA5/era5_uwnd_2023.nc'
vwnd_nc_file_path = '../data/raw/reanalyses/ERA5/era5_vwnd_2023.nc'
try:
    tree, valid_time_dt, latitudes, longitudes, lat_lon_pairs = precompute_kdtree_and_time_diffs(uwnd_nc_file_path)
except Exception as e:
    print(f"Error precomputing KDTree and time differences: {e}")
    raise


Precomputing KDTree and time differences...
KDTree and time differences precomputed successfully.


Function to extract wind components at a given lat/lon (preloads reanalysis netCDFs also)

In [4]:
uwnd_nc_file_path = '../data/raw/reanalyses/ERA5/era5_uwnd_2023.nc'
vwnd_nc_file_path = '../data/raw/reanalyses/ERA5/era5_vwnd_2023.nc'

uwnd_ds = nc.Dataset(uwnd_nc_file_path)
vwnd_ds = nc.Dataset(vwnd_nc_file_path)

uwnd_array = uwnd_ds.variables['u'][:, 0, :, :]  # Assuming 'u' is the variable name for u-component wind and removing the pressure dimension
vwnd_array = vwnd_ds.variables['v'][:, 0, :, :]  # Assuming 'v' is the variable name for v-component wind and removing the pressure dimension

# Function to extract wind components
def extract_wind_components(lat, lon, dt, tree, valid_time_dt, latitudes, longitudes, lat_lon_pairs):
    try:
        # Convert the given datetime to a numpy datetime64 object
        row_datetime = np.datetime64(dt)

        # Find the value in the valid_time dimension closest in time to the datetime in the dataframe
        time_diffs = np.abs(valid_time_dt - row_datetime)
        closest_time_index = np.argmin(time_diffs)

        # Check if the calculated index is within the bounds of the uwnd_array
        if closest_time_index < 0 or closest_time_index >= uwnd_array.shape[0]:
            raise ValueError("The given datetime is out of bounds for the NetCDF data")

        # Select the corresponding netCDF slices
        uwnd_slice = uwnd_array[closest_time_index, :, :]
        vwnd_slice = vwnd_array[closest_time_index, :, :]

        # Find the grid cell of the netCDF slice closest to the given Latitude and Longitude position
        lat_lon = (lat, lon)
        _, closest_point_index = tree.query(lat_lon)
        closest_lat, closest_lon = lat_lon_pairs[closest_point_index]

        # Find the index of the closest latitude/longitude pair in the arrays
        lat_index = np.where(latitudes == closest_lat)[0][0]
        lon_index = np.where(longitudes == closest_lon)[0][0]

        # Extract the u and v wind components
        u_wind = uwnd_slice[lat_index, lon_index]
        v_wind = vwnd_slice[lat_index, lon_index]

        # Round wind components to 4 decimal places
        u_wind = round(u_wind, 4)
        v_wind = round(v_wind, 4)

        return u_wind, v_wind
    except Exception as e:
        print(f"Error extracting wind components: {e}")
        raise

Function to calculate new position from current position, displacement, and heading

In [5]:
def calculate_new_position(current_position, displacement, heading):
    R = 6371000  # Earth's radius in meters
    
    lat1 = math.radians(current_position[0])
    lon1 = math.radians(current_position[1])
    heading_rad = math.radians(heading)
    
    lat2 = math.asin(math.sin(lat1) * math.cos(displacement / R) +
                     math.cos(lat1) * math.sin(displacement / R) * math.cos(heading_rad))
    
    lon2 = lon1 + math.atan2(math.sin(heading_rad) * math.sin(displacement / R) * math.cos(lat1),
                             math.cos(displacement / R) - math.sin(lat1) * math.sin(lat2))
    
    return math.degrees(lat2), math.degrees(lon2)

## Model Optimization and Evaluation Functions

These functions handle model optimization with AutoML and evaluation of predictions. (implementation ongoing)

In [6]:
def evaluate_predictions(true_data, predicted_file):
    """
    Evaluate the accuracy of predictions against true data.
    """
    # Read predicted data
    pred_data = pd.read_csv(predicted_file)
    pred_data['Datetime'] = pd.to_datetime(pred_data['Datetime'])
    
    # Merge true and predicted data on datetime
    merged_data = pd.merge(
        true_data,
        pred_data,
        left_on=['datetime'],
        right_on=['Datetime'],
        suffixes=('_true', '_pred')
    )
    
    # Calculate position errors
    position_errors = []
    for _, row in merged_data.iterrows():
        true_pos = (row['Latitude_true'], row['Longitude_true'])
        pred_pos = (row['Latitude'], row['Longitude'])
        error_km = haversine(true_pos, pred_pos)
        position_errors.append(error_km)
    
    merged_data['position_error_km'] = position_errors
    
    # Calculate metrics
    metrics = {
        'mean_position_error_km': np.mean(position_errors),
        'median_position_error_km': np.median(position_errors),
        'max_position_error_km': np.max(position_errors),
        'std_position_error_km': np.std(position_errors),
        'rmse_lat': np.sqrt(mean_squared_error(merged_data['Latitude_true'], merged_data['Latitude'])),
        'rmse_lon': np.sqrt(mean_squared_error(merged_data['Longitude_true'], merged_data['Longitude'])),
        'mae_lat': mean_absolute_error(merged_data['Latitude_true'], merged_data['Latitude']),
        'mae_lon': mean_absolute_error(merged_data['Longitude_true'], merged_data['Longitude'])
    }
    
    return metrics, merged_data

def plot_trajectory_comparison(merged_data, buoy_id, model_name):
    """
    Plot true vs predicted trajectories
    """
    plt.figure(figsize=(12, 8))
    plt.plot(merged_data['Longitude_true'], merged_data['Latitude_true'], 
             'b-', label='True Trajectory')
    plt.plot(merged_data['Longitude'], merged_data['Latitude'], 
             'r--', label='Predicted Trajectory')
    plt.title(f'True vs Predicted Trajectory - Buoy {buoy_id}\nModel: {model_name}')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.legend()
    plt.grid(True)
    
    # Save plot
    plt.savefig(f'../data/processed/predictions/trajectory_comparison_{buoy_id}_{model_name}.png')
    plt.close()

def objective(trial, X_train, y_train):
    """
    Optuna objective function for hyperparameter optimization
    """
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 10, 100),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2']),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False])
    }
    
    model = RandomForestRegressor(**params, n_jobs=-1)
    scores = cross_val_score(
        model, X_train, y_train, 
        cv=3, scoring='neg_mean_squared_error'
    )
    
    return -np.mean(scores)

def train_optimized_model(X_train, y_train, n_trials=100):
    """
    Train model with optimized hyperparameters using Optuna
    """
    study = optuna.create_study(direction='minimize')
    study.optimize(
        lambda trial: objective(trial, X_train, y_train),
        n_trials=n_trials
    )
    
    best_params = study.best_params
    print("Best parameters:", best_params)
    
    best_model = RandomForestRegressor(**best_params, n_jobs=-1)
    best_model.fit(X_train, y_train)
    
    return best_model, best_params

def evaluate_all_predictions(val_data, predictions_dir):
    """
    Evaluate all prediction files in the specified directory
    """
    results = []
    prediction_files = glob.glob(f"{predictions_dir}/predicted_*.csv")
    
    for pred_file in prediction_files:
        # Extract buoy_id and model_name from filename
        filename = pred_file.split('/')[-1]
        buoy_id = filename.split('_')[1]
        model_name = filename.split('_')[2].replace('.csv', '')
        
        # Get true data for this buoy
        true_data_buoy = val_data[val_data['BuoyID'] == int(buoy_id)]
        
        # Calculate metrics
        metrics, merged_data = evaluate_predictions(true_data_buoy, pred_file)
        metrics['buoy_id'] = buoy_id
        metrics['model_name'] = model_name
        
        # Plot trajectory comparison
        plot_trajectory_comparison(merged_data, buoy_id, model_name)
        
        results.append(metrics)
    
    # Convert results to DataFrame
    results_df = pd.DataFrame(results)
    
    # Save results
    results_df.to_csv(f'{predictions_dir}/evaluation_results.csv', index=False)
    
    return results_df

Iterative predictor function

In [11]:
def iterative_prediction(val_data, model, tree, valid_times, latitudes, longitudes, lat_lon_pairs):
    start_time = time.time()
    max_duration = 4 * 60 * 60  # Maximum runtime in seconds

    # Initialize an empty list to store predictions for all buoys
    all_predictions = []

    # Iterate over each unique BuoyID
    unique_buoy_ids = val_data['BuoyID'].unique()
    for buoy_id in unique_buoy_ids:
        print(f"Processing BuoyID: {buoy_id}")
        buoy_data = val_data[val_data['BuoyID'] == buoy_id]

        # Initialize an empty list to store predictions for the current buoy
        predictions = []

        # Extract initial conditions for the current buoy
        current_lat, current_lon = buoy_data.iloc[0][['Latitude', 'Longitude']]
        current_uwnd, current_vwnd = buoy_data.iloc[0][['era5_uwnd', 'era5_vwnd']]

        for i in range(1, len(buoy_data)):
            # Check if the maximum duration has been exceeded
            elapsed_time = time.time() - start_time
            if elapsed_time > max_duration:
                print("Maximum duration exceeded. Stopping the script.")
                break

            next_row = buoy_data.iloc[i]

            input_data = pd.DataFrame({
                'Latitude': [current_lat],
                'Longitude': [current_lon],
                'era5_uwnd': [current_uwnd],
                'era5_vwnd': [current_vwnd]
            })

            # Make prediction for displacement and heading
            prediction_start_time = time.time()
            predicted_displacement, predicted_heading = model.predict(input_data)[0]
            predicted_lat, predicted_lon = calculate_new_position(
                (current_lat, current_lon),
                predicted_displacement,
                predicted_heading
            )
            prediction_end_time = time.time()

            # Extract wind components at the predicted position and time
            wind_extraction_start_time = time.time()
            predicted_wind_u, predicted_wind_v = extract_wind_components(
                predicted_lat, 
                predicted_lon, 
                next_row['datetime'],
                tree,
                valid_times,
                latitudes,
                longitudes,
                lat_lon_pairs
            )
            wind_extraction_end_time = time.time()

            # Append the prediction for the current buoy
            predictions.append([
                predicted_lat,
                predicted_lon,
                next_row['datetime']
            ])

            # Update current state for the next iteration
            current_lat, current_lon = predicted_lat, predicted_lon
            current_uwnd, current_vwnd = predicted_wind_u, predicted_wind_v

        # Append predictions of the current buoy to all_predictions
        all_predictions.extend(predictions)

    # Convert all predictions to a NumPy array before returning
    all_predictions_array = np.array(all_predictions, dtype=object)
    print(f"\nPrediction complete. Total predictions: {len(all_predictions)}")

    return all_predictions_array


Model selection, training, and validation (includes computation timing calculation)

In [14]:
from sklearn.model_selection import GroupKFold
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import time  # To track timing

# Load the data from the spreadsheet
buoy_data = pd.read_csv('../combined_buoy_data.csv')

# Drop unused columns
columns_to_keep = ['Latitude', 'Longitude', 'BuoyID', 'datetime', 'era5_uwnd', 'era5_vwnd', 'displacement', 'heading']
print("Dropping unused columns and retaining only:", columns_to_keep)
buoy_data = buoy_data[columns_to_keep].copy()
buoy_data['datetime'] = pd.to_datetime(buoy_data['datetime'])
print("Datetime column successfully converted to datetime format.")

# Define features and targets
X = buoy_data[['Latitude', 'Longitude', 'era5_uwnd', 'era5_vwnd', 'BuoyID', 'datetime']]
y = buoy_data[['displacement', 'heading']]
groups = buoy_data['BuoyID']

# Models to evaluate with proper multi-output handling
model_configs = [
     ('ElasticNet', MultiOutputRegressor(ElasticNet(alpha=1.0, l1_ratio=0.5))),
    ('GradientBoosting', MultiOutputRegressor(GradientBoostingRegressor(n_estimators=100, max_depth=5))),
    ('RandomForest', RandomForestRegressor(n_estimators=100, max_depth=10)),
    ('XGBoost', MultiOutputRegressor(XGBRegressor(n_estimators=100, max_depth=6, objective='reg:squarederror'))),
    ('LightGBM', MultiOutputRegressor(lgb.LGBMRegressor(n_estimators=100, max_depth=6)))
]

# GroupKFold for cross-validation
cv_folds = 5
group_kf = GroupKFold(n_splits=cv_folds)

# Dictionary to store cross-validation results and timing
cv_results = {}
timing_results = {}

# Iterate over each model
for model_name, model in model_configs:
    print(f"Testing model: {model_name}")
    
    model_scores = []
    fold_times = []

    # Perform GroupKFold cross-validation
    for fold_num, (train_index, val_index) in enumerate(group_kf.split(X, y, groups=groups)):
        fold_buoys = buoy_data.iloc[val_index]['BuoyID'].nunique()
        print(f"Fold {fold_num + 1}: {fold_buoys} unique buoys in validation set")
        
        # Split the data into training and validation
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        # Retain 'BuoyID' in X_val for iteration step
        X_val_with_buoyid = X_val.copy()
        X_train = X_train.drop(columns=['BuoyID', 'datetime'])  # Exclude 'BuoyID' for training
        X_val = X_val.drop(columns=['BuoyID', 'datetime'])  # Exclude 'BuoyID' for validation features

        # Measure training and validation time
        start_time = time.time()
        print(f"Training model on fold {fold_num+1}...")
        model.fit(X_train, y_train)
        print(f"Model {model_name} trained on Fold {fold_num+1}")
        
        # Make iterative predictions, using X_val_with_buoyid
        y_pred = iterative_prediction(
            val_data=X_val_with_buoyid,  # Pass full data with 'BuoyID' to iterator
            model=model, 
            tree=tree, 
            valid_times=valid_time_dt, 
            latitudes=latitudes, 
            longitudes=longitudes, 
            lat_lon_pairs=lat_lon_pairs
        )

        # Calculate RMSE for this fold
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        model_scores.append(rmse)
        print(f"Fold {fold_num+1} RMSE: {rmse:.3f}")

        # Record the time taken for this fold
        fold_time = time.time() - start_time
        fold_times.append(fold_time)
        print(f"Fold {fold_num+1} time taken: {fold_time:.2f} seconds")
    
    # Store cross-validation results
    cv_results[model_name] = model_scores
    timing_results[model_name] = fold_times
    print(f"\nCross-validation completed for {model_name}. "
          f"Mean RMSE: {np.mean(model_scores):.3f}, Std. Dev: {np.std(model_scores):.3f}\n"
          f"Mean time: {np.mean(fold_times):.2f} seconds, Total time: {np.sum(fold_times):.2f} seconds\n")

# Select the best model based on mean RMSE
best_model_name = min(cv_results, key=lambda k: np.mean(cv_results[k]))
best_model = dict(model_configs)[best_model_name]
print(f"\n=== Best model selected: {best_model_name} with mean RMSE: {np.mean(cv_results[best_model_name]):.3f} ===")

Dropping unused columns and retaining only: ['Latitude', 'Longitude', 'BuoyID', 'datetime', 'era5_uwnd', 'era5_vwnd', 'displacement', 'heading']
Datetime column successfully converted to datetime format.
Testing model: ElasticNet
Fold 1: 47 unique buoys in validation set
Training model on fold 1...
Model ElasticNet trained on Fold 1
Processing BuoyID: 900124
Processing BuoyID: 900130
Processing BuoyID: 900138
Processing BuoyID: 902005
Processing BuoyID: 300025060513230
Processing BuoyID: 300234060321940
Processing BuoyID: 300234060330560
Processing BuoyID: 300234060727760
Processing BuoyID: 300234060728940
Processing BuoyID: 300234061164500
Processing BuoyID: 300234066088170
Processing BuoyID: 300234066534020
Processing BuoyID: 300234066691600
Processing BuoyID: 300234066797650
Processing BuoyID: 300234066891240
Processing BuoyID: 300234066892270
Processing BuoyID: 300234066893240
Processing BuoyID: 300234066894280
Processing BuoyID: 300234067977270
Processing BuoyID: 300234068242250
P

ValueError: Found input variables with inconsistent numbers of samples: [319758, 319711]

## Model Optimization with AutoML

Hyperparameter tuning on the best model with Optuna

In [9]:
# Train optimized model
print("Training optimized model with Optuna...")
best_model, best_params = train_optimized_model(X_train, y_train, n_trials=100)
print("\nBest parameters found:", best_params)

[I 2024-11-21 20:35:09,247] A new study created in memory with name: no-name-d4c3eda4-6395-409f-85c0-8ffcf1f7d5bc


Training optimized model with Optuna...


[I 2024-11-21 20:37:27,612] Trial 0 finished with value: 1016308.9119413403 and parameters: {'n_estimators': 404, 'max_depth': 28, 'min_samples_split': 15, 'min_samples_leaf': 8, 'max_features': 'log2', 'bootstrap': True}. Best is trial 0 with value: 1016308.9119413403.
[I 2024-11-21 20:44:49,922] Trial 1 finished with value: 1126776.3119643775 and parameters: {'n_estimators': 914, 'max_depth': 63, 'min_samples_split': 15, 'min_samples_leaf': 8, 'max_features': 'sqrt', 'bootstrap': False}. Best is trial 0 with value: 1016308.9119413403.
[I 2024-11-21 20:49:13,119] Trial 2 finished with value: 1187049.5309834296 and parameters: {'n_estimators': 520, 'max_depth': 43, 'min_samples_split': 4, 'min_samples_leaf': 7, 'max_features': 'sqrt', 'bootstrap': False}. Best is trial 0 with value: 1016308.9119413403.
[I 2024-11-21 20:53:36,569] Trial 3 finished with value: 1188383.1698805906 and parameters: {'n_estimators': 625, 'max_depth': 18, 'min_samples_split': 10, 'min_samples_leaf': 6, 'max_fe

ValueError: 
All the 3 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\benem\anaconda3\envs\mlggeo2024_aobuoypredict\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\benem\anaconda3\envs\mlggeo2024_aobuoypredict\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "c:\Users\benem\anaconda3\envs\mlggeo2024_aobuoypredict\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "c:\Users\benem\anaconda3\envs\mlggeo2024_aobuoypredict\Lib\site-packages\sklearn\utils\_param_validation.py", line 95, in 

Training on the entire dataset with tuned best model

In [ ]:

# Train the best model on the entire training dataset
print("Training best model...")
best_model.fit(X_train, y_train)
print("Model training complete.")

# Ensure the predictions directory exists
predictions_dir = '../data/predictions'
if not os.path.exists(predictions_dir):
    os.makedirs(predictions_dir)
    print(f"Directory {predictions_dir} created.")
else:
    print(f"Directory {predictions_dir} already exists.")

# Get unique BuoyIDs
unique_buoy_ids = val_data['BuoyID'].unique()

# Iterate over each BuoyID in the validation data and make predictions
for buoy_id in unique_buoy_ids:
    # Set output file path
    output_file_path = f'../data/predictions/predictions_{buoy_id}.csv'
    
    # Subset the data for the current BuoyID
    buoy_data = val_data[val_data['BuoyID'] == buoy_id]

    # Run iterative predictions on validation subset
    print("\nStarting iterative predictions on validation subset...")
    iterative_prediction(
        val_data=buoy_data,
        model=best_model,
        tree=tree,
        valid_times=valid_time_dt,
        latitudes=latitudes,
        longitudes=longitudes,
        lat_lon_pairs=lat_lon_pairs,
        output_file_path=output_file_path
    )







## Prediction Evaluation

In [ ]:
# Evaluate all predictions
print("Evaluating predictions...")
results_df = evaluate_all_predictions(val_data, '../data/processed/predictions/')

# Display summary statistics
print("\nSummary Statistics:")
print("\nMean metrics across all buoys:")
print(results_df.mean(numeric_only=True))
print("\nMetrics by model:")
print(results_df.groupby('model_name').mean(numeric_only=True))

# Plot overall error distribution
plt.figure(figsize=(10, 6))
plt.hist(results_df['mean_position_error_km'], bins=20)
plt.title('Distribution of Mean Position Errors')
plt.xlabel('Mean Position Error (km)')
plt.ylabel('Frequency')
plt.savefig('../data/processed/predictions/error_distribution.png')
plt.close()

